In [56]:
%reload_ext autoreload
%autoreload 2

import os, sys
import pickle

import numpy as np
from scipy import interpolate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [2]:
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm'
sys.path.append(os.environ['DH_DIR'])
sys.path.append('..')

In [3]:
import main

import dm21cm.physics as phys

## config

In [60]:
run_name = 'ee_21reion'

# DM turns on
#injection_modifier_func = lambda rs: float(rs < 41)
injection_modifier_func = None

# custom xe levels
base_records = np.load(f'../data/run_info/base_records.npy', allow_pickle=True).item()
def x_reion_func(rs):
    # extrapolate z low so that darkhistory doesn't complain
    z_arr = base_records['z'].copy()
    z_arr = np.append(z_arr, 2 * z_arr[-1] - z_arr[-2])
    x_e_arr = base_records['x_e'].copy()
    x_e_arr = np.append(x_e_arr, x_e_arr[-1])
    return interpolate.interp1d(1 + z_arr, x_e_arr)(rs)

reion_rs = 41
end_rs = base_records['z'][-1] + 1

In [61]:
soln = main.evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='elec_delta',
    start_rs=3000, end_rs=end_rs,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1, injection_modifier_func=injection_modifier_func,
    xe_reion_func=x_reion_func, reion_rs=reion_rs, use_tqdm=False
)

Loading time: 0.000 s
Initialization time: 0.191 s


/zfs/yitians/anaconda3/lib/python3.8/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Main loop time: 148.839 s


In [63]:
pickle.dump(soln, open(f'../data/run_info/DH/{run_name}_soln.p', 'wb'))

In [64]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]

In [65]:
np.save(f'../data/run_info/DH/{run_name}_records', arr_records)